In [38]:
import pandas as pd
import numpy as np

cols = set(np.arange(44))
irrelevant = set([0, 1, 2, 7, 8, 9, 10, 17, 22, 23, 24, 25, 26, 27, 28, 
                  29, 30, 31, 32, 33, 35, 36, 37, 38, 39, 40, 41, 42, 43])
useful_cols = list(cols.difference(irrelevant))
df = pd.read_excel('menustat_2021_dataset.xlsx', usecols=useful_cols)
reviews = pd.read_csv('reviews.csv')

In [39]:
df = df[~(df['limited_time_offer'] == 1)]
df.drop('limited_time_offer', axis=1, inplace=True)
df = df[(df['food_category'] != 'Toppings & Ingredients') & (df['food_category'] != 'Beverages')]
df = df[~(df['restaurant'] == 'Chick-Fil-A Lemonade, Medium')]
df.count()

food_category       14248
restaurant          14248
item_name           14248
item_description    14248
calories            13851
total_fat           13831
saturated_fat       13765
trans_fat           13493
cholesterol         13697
sodium              13875
carbohydrates       13829
protein             13842
sugar               13569
dietary_fiber       13315
dtype: int64

In [ ]:
import requests
from bs4 import SoupStrainer, BeautifulSoup as bs
from bs4 import SoupStrainer
import re

def get_restaurant_links(desc, loc="nyc"):
    strainer = SoupStrainer("a", attrs={"href": True})
    url = f'https://www.yelp.com/search?find_desc={desc}&find_loc={loc}&sortby=review_count'
    page = requests.get(url)
    soup = bs(page.content, "html.parser", parse_only=strainer)
    links = [link.get('href') for link in soup]
    url_desc = desc.lower().split(' ')[0].replace("'", '')
    regex = f'/biz/{url_desc}.*'
    filtered = filter(lambda link : re.match(regex, link) and not 
    re.match(r'.*?hrid.*', link), links)
    full_links = map(lambda link : f"https://www.yelp.com{link}", filtered)
    return set(full_links)

dtypes = np.dtype([("restaurant", str), ("rating", int), ("review", str)])
states = ['new york', 'california', 'georgia', 'texas', 'minnesota', 'missouri']
reviews = pd.DataFrame(np.empty(0, dtype=dtypes))
restaurants = df['restaurant'].unique()
for restaurant in restaurants:
    review_count = 0
    for state in states:
        if review_count > 30:
            break
        for url in get_restaurant_links(restaurant, state):
            strainer = SoupStrainer("ul")
            soup = bs(requests.get(url).content, "html.parser", parse_only=strainer)
            for div in soup.find_all('div'):
                rating = div.find('div', attrs = {'role': 'img', 'aria-label': True})
                para = div.find('p')
                if para and rating and para.span:
                    rating = rating['aria-label']
                    rating = int(rating[:rating.find(' ')])
                    reviews = reviews.append({"restaurant": restaurant, "rating": rating, "review": para.text}, ignore_index=True)
                    review_count += 1
reviews.count()

In [40]:
def add_quote(string):
    return '"' + string.strip() + '"'
for text_field in ['food_category', 'restaurant', 'item_name', 'item_description']:
    df[text_field] = df[text_field][~df[text_field].isna()].astype(str).map(add_quote)
df = df[df['restaurant'].isin(reviews['restaurant'].unique())]
df.count()

food_category       13514
restaurant          13514
item_name           13514
item_description    13514
calories            13117
total_fat           13097
saturated_fat       13031
trans_fat           12769
cholesterol         12969
sodium              13141
carbohydrates       13097
protein             13111
sugar               12848
dietary_fiber       12621
dtype: int64

In [41]:
reviews.to_csv("reviews.csv", index=False, encoding='utf-8')
df.to_csv('fastfood.csv', index=False, encoding='utf-8') 